<a href="https://colab.research.google.com/github/joel280584/PineConeLearning/blob/main/Query_IMDB_DB_for_movie_list_using_MiniLM_L12_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Query IMDB DB for movie list

In [1]:
!pip install pinecone
!pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.8/419.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 4.5 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
apikey = userdata.get('API_KEY')
from pinecone import Pinecone, ServerlessSpec
pc= Pinecone(api_key=apikey)

In [3]:
pc.list_indexes()
pc.delete_index("quickstart")
pc.list_indexes()

[]

In [4]:
index_name = "quickstart"

pc.create_index(
    name=index_name,
    dimension=384, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

In [5]:
#pc.delete_index(index_name)
pc.list_indexes()

[
    {
        "name": "quickstart",
        "dimension": 384,
        "metric": "cosine",
        "host": "quickstart-2s9kc7h.svc.aped-4627-b74a.pinecone.io",
        "spec": {
            "serverless": {
                "cloud": "aws",
                "region": "us-east-1"
            }
        },
        "status": {
            "ready": true,
            "state": "Ready"
        },
        "deletion_protection": "disabled"
    }
]

In [6]:
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [7]:
imdb_df = pd.read_csv('/content/drive/MyDrive/Untitled folder/imdb_movies.csv')
imdb_df.head()

,names,date_x,score,genre,overview,crew,orig_title,status,orig_lang,budget_x,revenue,country
0,Creed III,03/02/2023,73.0,"Drama, Action","After dominating the boxing world, Adonis Cree...","Michael B. Jordan, Adonis Creed, Tessa Thompso...",Creed III,Released,English,75000000.0,2.716167e+08,AU
1,Avatar: The Way of Water,12/15/2022,78.0,"Science Fiction, Adventure, Action",Set more than a decade after the events of the...,"Sam Worthington, Jake Sully, Zoe Saldaña, Neyt...",Avatar: The Way of Water,Released,English,460000000.0,2.316795e+09,AU
2,The Super Mario Bros. Movie,04/05/2023,76.0,"Animation, Adventure, Family, Fantasy, Comedy","While working underground to fix a water main,...","Chris Pratt, Mario (voice), Anya Taylor-Joy, P...",The Super Mario Bros. Movie,Released,English,100000000.0,7.244590e+08,AU
3,Mummies,01/05/2023,70.0,"Animation, Comedy, Family, Adventure, Fantasy","Through a series of unfortunate events, three ...","Óscar Barberán, Thut (voice), Ana Esther Albor...",Momias,Released,"Spanish, Castilian",12300000.0,3.420000e+07,AU
4,Supercell,03/17/2023,61.0,Action,Good-hearted teenager William always lived in ...,"Skeet Ulrich, Roy Cameron, Anne Heche, Dr Quin...",Supercell,Released,English,77000000.0,3.409420e+08,US


In [8]:
imdb_df.shape

(10178, 12)

In [9]:
imdb_df.isna().sum()
imdb_df.reset_index(drop=True)
imdb_df.shape
imdb_df.iloc[[10051]]


,names,date_x,score,genre,overview,crew,orig_title,status,orig_lang,budget_x,revenue,country
10051,Cabin Fever,02/12/2016,47.0,Horror,"In this grisly remake of the 2002 horror hit, ...","Eli Roth, , Teresa Decher, Emily, Samuel Davis...",Cabin Fever,Released,English,1500000.0,30351664.0,US


In [10]:
imdb_df.dropna(inplace=True)

In [12]:
#imdb_df['context2vectorize']= 'Name:'+ imdb_df['names'] + '. Genre '+ imdb_df['genre'] + '. ReleaseDate: '+ imdb_df['date_x']+ '. Overview:'+ imdb_df['overview'] + '. Crew: '+ imdb_df['crew']
imdb_df['context2vectorize']= 'Genre: '+ imdb_df['genre'] + '. ReleaseDate: '+ imdb_df['date_x']+ '. Overview:'+ imdb_df['overview']

In [13]:
imdb_df['context2vectorize'].head()

,context2vectorize
0,"Genre: Drama, Action. ReleaseDate: 03/02/2023 ..."
1,"Genre: Science Fiction, Adventure, Action. Rel..."
2,"Genre: Animation, Adventure, Family, Fantasy, ..."
3,"Genre: Animation, Comedy, Family, Adventure, F..."
4,Genre: Action. ReleaseDate: 03/17/2023 . Overv...


In [14]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [15]:
imdb_df['vector'] = imdb_df['context2vectorize'].apply(lambda sentence: model.encode(sentence))

In [16]:
imdb_df['id'] = imdb_df.index.astype(str)
imdb_df['values'] = imdb_df['vector'].tolist()
imdb_df['metadata'] = imdb_df.apply(lambda row: {'names':row['names'], 'genre':row['genre'],
                                                   'date_x':row['date_x'], 'crew':row['crew'],}, axis=1)

In [17]:
imdb_df['metadata']

,metadata
0,"{'names': 'Creed III', 'genre': 'Drama, Action..."
1,"{'names': 'Avatar: The Way of Water', 'genre':..."
2,"{'names': 'The Super Mario Bros. Movie', 'genr..."
3,"{'names': 'Mummies', 'genre': 'Animation, Come..."
4,"{'names': 'Supercell', 'genre': 'Action', 'dat..."
...,...
10173,"{'names': '20th Century Women', 'genre': 'Dram..."
10174,{'names': 'Delta Force 2: The Colombian Connec...
10175,"{'names': 'The Russia House', 'genre': 'Drama,..."
10176,"{'names': 'Darkman II: The Return of Durant', ..."


In [18]:
upsert_df = imdb_df[['id','values','metadata']]

In [19]:
upsert_df.head()

,id,values,metadata
0,0,"[-0.08079438, 0.0368194, -0.06558993, -0.04603...","{'names': 'Creed III', 'genre': 'Drama, Action..."
1,1,"[-0.060387906, -0.029802473, -0.09579538, -0.0...","{'names': 'Avatar: The Way of Water', 'genre':..."
2,2,"[-0.0745932, -0.03028558, 0.057697978, -0.0525...","{'names': 'The Super Mario Bros. Movie', 'genr..."
3,3,"[-0.05706818, 0.0037008454, 0.037306033, 0.011...","{'names': 'Mummies', 'genre': 'Animation, Come..."
4,4,"[-0.06613365, 0.0018621939, -0.044634815, -0.0...","{'names': 'Supercell', 'genre': 'Action', 'dat..."


In [20]:
index = pc.Index("quickstart")
index.upsert_from_dataframe(upsert_df, namespace="quickstart")

sending upsert requests:   0%|          | 0/10052 [00:00<?, ?it/s]

{'upserted_count': 10052}

In [21]:
def query_pinecone(query, top_k):
  xc = index.query(vector = (model.encode(query)).tolist(), # python list
           top_k=10, namespace="quickstart",include_metadata=True)
  print_df = pd.DataFrame()
  temp_df = pd.DataFrame()
  for ids in xc.matches:
    print(ids.id,":", ids.score)
    temp_df= imdb_df[imdb_df['id'] == ids.id]
    print_df = pd.concat([print_df, imdb_df[imdb_df['id'] == ids.id]])
  return print_df

In [22]:
query_pinecone("List Animation genre and was released in the 2023", 10)

4965 : 0.705196142
8222 : 0.657532
784 : 0.632308066
3940 : 0.629629314
6309 : 0.628263593
3936 : 0.625709176
4500 : 0.610265553
4400 : 0.591838419
3944 : 0.589260757
4363 : 0.58344835


,names,date_x,score,genre,overview,crew,orig_title,status,orig_lang,budget_x,revenue,country,context2vectorize,vector,id,values,metadata
4965,Irmão do Jorel - Especial Carnaval Bruttal,02/25/2022,63.0,"Animation, Comedy, Family",We don't have an overview translated in Englis...,"Andrei Duarte, Jorel's Brother (voice)",Irmão do Jorel - Especial Carnaval Bruttal,Released,Portuguese,115000000.0,355548006.2,BR,"Genre: Animation, Comedy, Family. ReleaseDate:...","[-0.043418575, -0.077435054, 0.01694832, -0.04...",4965,"[-0.043418575, -0.077435054, 0.01694832, -0.04...",{'names': 'Irmão do Jorel - Especial Carnaval ...
8222,K: Seven Stories Movie 6 - Circle Vision - Nam...,12/01/2018,65.0,"Animation, Action, Fantasy",We don't have an overview translated in Englis...,"Daisuke Namikawa, Yashiro Isana, Daisuke Ono, ...",K Seven Stories Circle Vision ~Nameless Song~,Released,Japanese,141280000.0,608857282.0,JP,"Genre: Animation, Action, Fantasy. ReleaseDate...","[-0.00681383, -0.05679232, 0.030081548, -0.034...",8222,"[-0.00681383, -0.05679232, 0.030081548, -0.034...",{'names': 'K: Seven Stories Movie 6 - Circle V...
784,The House,01/14/2022,71.0,"Animation, Drama, Comedy, Horror, Fantasy","Across different eras, a poor family, an anxio...","Mia Goth, Mabel (voice), Matthew Goode, Raymon...",The House,Released,English,40000000.0,31192425.0,GB,"Genre: Animation, Drama, Comedy, Horror, Fanta...","[-0.02856195, -0.066741176, 0.015456325, -0.02...",784,"[-0.02856195, -0.066741176, 0.015456325, -0.02...","{'names': 'The House', 'genre': 'Animation, Dr..."
3940,Kuroko's Basketball - Movie: Winter Cup - Cros...,12/03/2016,75.0,Animation,The third of three Kuroko no Basket compilatio...,"Kensho Ono, Tetsuya Kuroko (voice), Yuki Ono, ...",劇場版「黒子のバスケ」ウインターカップ総集編～扉の向こう～,Released,Japanese,99600000.0,396508441.0,JP,Genre: Animation. ReleaseDate: 12/03/2016 . Ov...,"[-0.07004161, -0.015036414, 0.019102972, -0.02...",3940,"[-0.07004161, -0.015036414, 0.019102972, -0.02...",{'names': 'Kuroko's Basketball - Movie: Winter...
6309,Naruto 20th Anniversary - Road of Naruto,10/02/2022,80.0,"Animation, Action",Promotional video celebrating 20 years of the ...,"Junko Takeuchi, Naruto Uzumaki (voice), Noriak...",Naruto 20th Anniversary - Road of Naruto,Released,Japanese,130000000.0,765868107.0,JP,"Genre: Animation, Action. ReleaseDate: 10/02/2...","[-0.122114494, 0.0046222126, 0.059685435, -0.0...",6309,"[-0.122114494, 0.0046222126, 0.059685435, -0.0...",{'names': 'Naruto 20th Anniversary - Road of N...
3936,Princess Principal Crown Handler: Chapter 3,04/07/2023,0.0,"Animation, Action, Adventure, Drama, Mystery",The third out of six movie sequels to the TV s...,"Aoi Koga, Ange (voice), Akira Sekine, Princess...",プリンセス・プリンシパル Crown Handler 第3章,Released,Japanese,231940000.0,178359863.0,JP,"Genre: Animation, Action, Adventure, Drama, My...","[-0.04665652, -0.06508597, -0.029857626, -0.04...",3936,"[-0.04665652, -0.06508597, -0.029857626, -0.04...",{'names': 'Princess Principal Crown Handler: C...
4500,The Little Mermaid Live!,11/05/2019,71.0,Music,Live musical performances by a star-studded ca...,"Auli'i Cravalho, Ariel, Edie McClurg, Carlotta...",The Little Mermaid Live!,Released,English,48300000.0,354965688.6,US,Genre: Music. ReleaseDate: 11/05/2019 . Overvi...,"[-0.0036011266, -0.076506905, 0.07919987, -0.0...",4500,"[-0.0036011266, -0.076506905, 0.07919987, -0.0...","{'names': 'The Little Mermaid Live!', 'genre':..."
4400,Koati,01/27/2022,73.0,Animation,"Three unlikely heroes - Nachi, a free-spirited...","Sofía Vergara, Zaina (voice), Sebastián Villal...",Koati,Released,English,123600000.0,590203752.2,AU,Genre: Animation. ReleaseDate: 01/27/2022 . Ov...,"[-0.10369418, -0.011014376, 0.010000538, -0.01...",4400,"[-0.10369418, -0.011014376, 0.010000538, -0.01...","{'names': 'Koati', 'genre': 'Animation', 'date..."
3944,Enthada Saji,04/08/2023,0.0,"Comedy, Fantasy, Drama",We don't have an overview translated in Englis...,"Kunchacko Boban, , Jayasurya, , Nivetha Th

In [23]:
query_pinecone("List Action genre and with released in 2023 ", 10)

4614 : 0.758465827
1293 : 0.747350156
3660 : 0.730586529
4647 : 0.725841224
3944 : 0.705076456
8222 : 0.699388683
5642 : 0.690725744
8994 : 0.690175772
4486 : 0.688122034
2948 : 0.687155128


,names,date_x,score,genre,overview,crew,orig_title,status,orig_lang,budget_x,revenue,country,context2vectorize,vector,id,values,metadata
4614,Ahimsa,04/07/2023,0.0,"Romance, Action, Drama",We don't have an overview translated in Englis...,"Abhiram Daggubati, Raghu, Geethika Tiwary, Aha...",టీజ‌ర్‌,Released,Telugu,223940000.0,175269998.8,IN,"Genre: Romance, Action, Drama. ReleaseDate: 04...","[-0.03458417, -0.07515589, 0.021537825, -0.040...",4614,"[-0.03458417, -0.07515589, 0.021537825, -0.040...","{'names': 'Ahimsa', 'genre': 'Romance, Action,..."
1293,Adanis: Kutsal Kavga,03/11/2022,51.0,"Drama, Action, Adventure",We don't have an overview translated in Englis...,"Esra Bilgiç, Cemre, İsmail Filiz, İlhan, Baki ...",Adanis: Kutsal Kavga,Released,Turkish,3740000.0,568828.0,TR,"Genre: Drama, Action, Adventure. ReleaseDate: ...","[-0.020964224, -0.057167776, 0.01688416, -0.02...",1293,"[-0.020964224, -0.057167776, 0.01688416, -0.02...","{'names': 'Adanis: Kutsal Kavga', 'genre': 'Dr..."
3660,Meg 2: The Trench,08/04/2023,0.0,"Science Fiction, Action, Horror, Thriller",We don't have an overview translated in Englis...,"Jason Statham, Jonas Taylor, Shuya Sophia Cai,...",Meg 2: The Trench,Post Production,English,231940000.0,178359863.0,US,"Genre: Science Fiction, Action, Horror, Thrill...","[0.0015887852, -0.07853697, 0.0077198036, -0.0...",3660,"[0.0015887852, -0.07853697, 0.0077198036, -0.0...","{'names': 'Meg 2: The Trench', 'genre': 'Scien..."
4647,Super Bodyguard,04/22/2021,60.0,"Action, Drama",We don't have an overview translated in Englis...,"Xie Mengwei, Ye Kai, Hao Shaowen, Tony, Cui Zh...",超神保镖,Released,Chinese,92200000.0,254504167.2,CN,"Genre: Action, Drama. ReleaseDate: 04/22/2021 ...","[-0.029235613, -0.0835528, 0.0103680985, -0.04...",4647,"[-0.029235613, -0.0835528, 0.0103680985, -0.04...","{'names': 'Super Bodyguard', 'genre': 'Action,..."
3944,Enthada Saji,04/08/2023,0.0,"Comedy, Fantasy, Drama",We don't have an overview translated in Englis...,"Kunchacko Boban, , Jayasurya, , Nivetha Thomas...",എന്താടാ സജി,Released,Malayalam,185000000.0,175269998.8,IN,"Genre: Comedy, Fantasy, Drama. ReleaseDate: 04...","[-0.018618846, -0.097557485, -0.0050512347, -0...",3944,"[-0.018618846, -0.097557485, -0.0050512347, -0...","{'names': 'Enthada Saji', 'genre': 'Comedy, Fa..."
8222,K: Seven Stories Movie 6 - Circle Vision - Nam...,12/01/2018,65.0,"Animation, Action, Fantasy",We don't have an overview translated in Englis...,"Daisuke Namikawa, Yashiro Isana, Daisuke Ono, ...",K Seven Stories Circle Vision ~Nameless Song~,Released,Japanese,141280000.0,608857282.0,JP,"Genre: Animation, Action, Fantasy. ReleaseDate...","[-0.00681383, -0.05679232, 0.030081548, -0.034...",8222,"[-0.00681383, -0.05679232, 0.030081548, -0.034...",{'names': 'K: Seven Stories Movie 6 - Circle V...
5642,Garota da Moto,09/23/2021,47.0,Action,We don't have an overview translated in Englis...,"Maria Casadevall, Joana, Kevin Vechiatto, Nico...",Garota da Moto,Released,Portuguese,129500000.0,847860141.2,BR,Genre: Action. ReleaseDate: 09/23/2021 . Overv...,"[-0.052214682, -0.03447413, 0.005725855, -0.05...",5642,"[-0.052214682, -0.03447413, 0.005725855, -0.05...","{'names': 'Garota da Moto', 'genre': 'Action',..."
8994,Странный дом,04/06/2023,0.0,"Science Fiction, Thriller",We don't have an overview translated in Englis...,"Vladimir Yaglych, , Ирина Кобзева, , Stanislav...",Странный дом,Released,Russian,228540000.0,175269998.8,RU,"Genre: Science Fiction, Thriller. ReleaseDate:...","[-0.022406276, -0.042850725, -0.0007066482, -0...",8994,"[-0.022406276, -0.042850725, -0.0007066482, -0...","{'names': 'Странный дом', 'genre': 'Science Fi..."
4486,Love You Long Time,04/08/2023,0.0,"Drama, Romance",We don't have an overview translated in Englis...,"Carlo Aquino, , Eisel Serrano, , Ana Abad-Sant...",Love You Long Time,Released,Tagalog,185534545.8,178359863.0,PH,"Genre: Drama, Romance. ReleaseDate: 04/08/2023...","[-0.036627926, -0.08250406, 0.022905568, -0.03...",4486,"[-